# 다양한 것

- Maximum Likelihood Estimation(MLE)
최대 가능 우도법 // 데이터가 다양한 분포중에서 해당 분포로 나왔을 가능도 $\\$
관측 데이터에 맞춰진 분포를 우도라고 함. 이렇게 찾아진 확률분포를 이용해서 Gradient Ascent(local maximum) & Grdient Descent(local minimum) 를 구함$\\$




In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

1.train, test 데이터를 활용한 softmax함수 성능평가

In [2]:
#m=8개의 samples데이터와 in_feature=3를 이용해서 0/1/2로 분류하는 softmax모델 만들기
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
x_train.shape, y_train.shape

(torch.Size([8, 3]), torch.Size([8]))

In [3]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([1, 2, 0])

In [4]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        #in_feature=3 -> 3개로 분류
        self.linear = nn.Linear(3,3)
    
    def forward(self,x):
        return self.linear(x)

In [5]:
model = SoftmaxClassifier()
model

SoftmaxClassifier(
  (linear): Linear(in_features=3, out_features=3, bias=True)
)

In [6]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [7]:
#train데이터에 대해서는 과적합되지 않는 선에서 여러번 epoch를 수행하므로써
#model를 여러번학습시키므로써 적당히 분류기 모델을 찾는다
def train(model, optimizer , x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        prediction = model(x_train)

        cost = F.cross_entropy(prediction,y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [8]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)

    predicted_class = prediction.max(1)[1]
    print( predicted_class)
    correct_count = (predicted_class==y_test).sum()

    cost = F.cross_entropy(prediction,y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

In [9]:
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [10]:
test(model, optimizer, x_test, y_test)

tensor([1, 1, 1])
Accuracy: 33.333335876464844% Cost: 1.209368


2. 학습률 lr를 조정
learning rate가 너무 크면 diverge가 되면서 gradient=0으로 수렴하지 않고 오히려 발산해버린다
learning rate가 너무 작으면 gradient=0으로 가는 속도가 너무 느리다

In [11]:
model = SoftmaxClassifier()

In [12]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [13]:
#learing rate가 너무 커섯 cost가 점점 커지는 현상(overshooting)
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.812500
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.000000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.250000
Epoch    7/20 Cost: 1532260.125000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.625000


In [14]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [15]:
#learning rate가 너무 작아서 cost가 거의 줄어들지 않는 현상
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


In [16]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [17]:
#적당한 learning rate를 찾아야됨
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131978
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


3. Data preprocessing(데이터 전처리)

In [18]:
#3개의 feature를 활용해서 예측값을 추출하는 다중선형회귀
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [19]:
m = x_train.mean(dim=0)
m

tensor([84.8000, 84.6000, 85.6000])

In [20]:
sigma = x_train.std(dim=0)
sigma

tensor([11.0544, 12.2393, 12.6214])

In [21]:
norm = (x_train-m)/sigma
norm

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])

In [22]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)

    def forward(self,x):
        return self.linear(x)

In [23]:
model = MultivariateLinearRegressionModel()
model

MultivariateLinearRegressionModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [24]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [25]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)

        cost = F.mse_loss(prediction,y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [26]:
#데이터 스케일링이 너무커서 cost를 올바르게 계산을 못함
train(model, optimizer, x_train,y_train)

Epoch    0/20 Cost: 57621.960938
Epoch    1/20 Cost: 1115690762240.000000
Epoch    2/20 Cost: 21603378185380036608.000000
Epoch    3/20 Cost: 418311281336289389130547200.000000
Epoch    4/20 Cost: 8099859627174812776678955834933248.000000
Epoch    5/20 Cost: inf
Epoch    6/20 Cost: inf
Epoch    7/20 Cost: inf
Epoch    8/20 Cost: inf
Epoch    9/20 Cost: inf
Epoch   10/20 Cost: inf
Epoch   11/20 Cost: inf
Epoch   12/20 Cost: nan
Epoch   13/20 Cost: nan
Epoch   14/20 Cost: nan
Epoch   15/20 Cost: nan
Epoch   16/20 Cost: nan
Epoch   17/20 Cost: nan
Epoch   18/20 Cost: nan
Epoch   19/20 Cost: nan


In [27]:
#스케일링한 데이터라서 잘 해결함
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train(model, optimizer, norm,y_train)

Epoch    0/20 Cost: 29476.677734
Epoch    1/20 Cost: 18728.175781
Epoch    2/20 Cost: 11946.346680
Epoch    3/20 Cost: 7634.267090
Epoch    4/20 Cost: 4882.764648
Epoch    5/20 Cost: 3124.194336
Epoch    6/20 Cost: 1999.400635
Epoch    7/20 Cost: 1279.729858
Epoch    8/20 Cost: 819.193237
Epoch    9/20 Cost: 524.461670
Epoch   10/20 Cost: 335.832794
Epoch   11/20 Cost: 215.106110
Epoch   12/20 Cost: 137.836304
Epoch   13/20 Cost: 88.378960
Epoch   14/20 Cost: 56.721436
Epoch   15/20 Cost: 36.456116
Epoch   16/20 Cost: 23.481792
Epoch   17/20 Cost: 15.174074
Epoch   18/20 Cost: 9.852999
Epoch   19/20 Cost: 6.443649


##3 overfitting : epoch를 많이 시도하면 train data에 대해서는 성능이 좋게 나오지만, $\\$
test data에서는 좋아지다가 일정시점이 된 후로는 다시 성능이 나빠짐.(성능의 기준은 cost를 기준으로)

-overfitting대처법
1. more data(많은 데이터를 활용)
2. less features(feature의 개수를 줄임)
3. Cross validation(교차 검정)
4. Regularization(규제)

4. Regularization
특정 가중치가 너무 크면 과대적합이 발생할 수 있음 => 가중치를 조정한다=모델을 규제하는 방식 : L1(라쏘), L2(릿지), 엘라스틱넷$\\$

-norm(벡터의 길이 or 크기를 측정하는 함수로 예측값-실제값의 거리르 재는 방식으로 사용된다) $\\$

$.\quad$ i) L1 Norm(Manhattan norm : 맨해튼 노름)
$.\quad$ 백터요소의 절댓값의 합 : L1 Norm = $\sum|x_i|$ $\\$ 제곱까지는 하지 않아서 이상치에 영향을 덜 받음 => Lasso(L1 규제)회귀에서 loss값을 규제할 때 이용$\\$
$.\quad$ ii) L2 Norm(Euclidean norm : 유클리리단 노름) 
$.\quad$ 피타고라스 정리를 이용한 최단거리 계산 : L2 Norm = $\sqrt(\sum x_i^2)$ $\\$제곱을 하므로 이상치에 영향을 받음 => Ridge(L2 규제)회귀에서 loss값을 규제할 때 이용$\\$

1. Lasso규제 = L1 Regularization : j(W) = MSE(W) + $a\sum |W_{i}|$ $\\$
L1규제를 Lasso규제라고 부르는데, 기존 비용함수 MSE에서 |가중치|를 더해서 비용함수를 계산하는 것을 말함 $\\$
Lasso규제의 목적은 중요하지 않은 가중치를 삭제하는 것이다. 즉, 가중치가 작은 W값들은 최적의 값을 찾아나가는 중에 점차 0이 되면서 아예 사라져버리게 된다$\\$
이렇게 되면 feature의 개수가 줄어들게 되어서 overfitting를 방지할 수 있다. $\\$
a : 모델을 얼마나 규제할 지 조정하는 하이퍼 파라미터이다. a가 크면 모든 가중치가 거의 0에 가깝게 설정이 가능하다.($\sum |W_{i}|$의 중요도가 훨씬 커지므로) 

2. Ridge규제 = L2 Regularization : j(W) = MSE(W) + $\frac{1}{2} a\sum (W_{i})^2$ $\\$ 
$\frac{1}{2}$ : 미분할 때 편하게 하기 위해 곱해줬다.$\\$
L2규제를 Ridge규제라고 부르는데, Lasso부분의 |가중치| -> $(가중치)^2$로 바뀐것이다$\\$
Ridge규제의 목적은 가중치가 너무 크지않은 방향으로 학습시키는 방식이다. $w^2$를 비용함수에 반영하다보니 너무 큰 W값들은 엄청 줄이게 만들 수 있음

In [28]:
#L1 norm, L2 norm, Lasso규제, Ridge규제를 직접 작성한 코드
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        
        #param은 각 feature의 가중치를 의미함
        #torch.norm(여러값들) : 텐서의 모든 값들을 제곱해서 더한후 제곱근(루트)를 씌운 하나의 scalar값
        for param in model.parameters():

            #L1 norm = L1 Regularization 사용 => |W|의 합
            print("L1 norm(L1 Regularization) : ",param,torch.norm(param,p=1),end='\n\n')

            #L2 norm => 피타고라스 최단거리
            print("L2 norm : ",param,torch.norm(param,p=2),end='\n\n')

            #L2 Regularization 사용 => |W|제곱의 합 = 위에서 구한 L2 norm의 제곱
            print("L2 Regularization : ",param,torch.norm(param,p=2)*torch.norm(param,p=2),end="\n\n----------------------------------------------------------------------------\n")

            #Regulaization을 사용할 때, L2 Norm를 사용하는 것도 이상한데, 
            #L2규제는 기존 MSE+가중치제곱의합을 더해주는건데,
            #여기서는 MSE+가중치제곱의합의 루트를 더해준다.
            #Regularization을 사용할건데, 왜 L2 norm를 갑자기 사용하냐?
            #https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-07_1_tips.ipynb
            l2_reg += torch.norm(param)
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        #print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch+1, nb_epochs, cost.item() ))

model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train_with_regularization(model, optimizer, norm, y_train)

L1 norm(L1 Regularization) :  Parameter containing:
tensor([[-0.0270, -0.3854,  0.3516]], requires_grad=True) tensor(0.7640, grad_fn=<NormBackward1>)

L2 norm :  Parameter containing:
tensor([[-0.0270, -0.3854,  0.3516]], requires_grad=True) tensor(0.5224, grad_fn=<NormBackward1>)

L2 Regularization :  Parameter containing:
tensor([[-0.0270, -0.3854,  0.3516]], requires_grad=True) tensor(0.2729, grad_fn=<MulBackward0>)

----------------------------------------------------------------------------
L1 norm(L1 Regularization) :  Parameter containing:
tensor([0.1792], requires_grad=True) tensor(0.1792, grad_fn=<NormBackward1>)

L2 norm :  Parameter containing:
tensor([0.1792], requires_grad=True) tensor(0.1792, grad_fn=<NormBackward1>)

L2 Regularization :  Parameter containing:
tensor([0.1792], requires_grad=True) tensor(0.0321, grad_fn=<MulBackward0>)

----------------------------------------------------------------------------
L1 norm(L1 Regularization) :  Parameter containing:
tensor([[